# Импорты

In [63]:
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import seaborn as sns
from math import exp, inf, tanh
from numpy import mean, std
from scipy.stats import norm

# Модели

In [64]:
class ChangePointSequence:
    def __init__(self, N, mean, standard_deviation, change_point_position):
        self.N = N - 1
        self.counter = -1
        self.mean = mean
        self.standard_deviation = standard_deviation
        self.change_point_position = change_point_position
        self.sequence = []
        
    def __iter__(self):
        return self
    
    def __next__(self):
        pass

    
class NoChangePoint(ChangePointSequence):
    '''
    Потоковая генерация случайной последовательности без разладки.
    '''
    def __init__(self, N, mean, standard_deviation):
        super().__init__(N, mean, standard_deviation, 0)
        
    def __next__(self):
        if self.counter < self.N:
            self.counter += 1
            new_element_sequence = float(norm.rvs(loc=self.mean, scale=self.standard_deviation, size=1))             
            self.sequence.append(new_element_sequence)
            return new_element_sequence
        else:
            raise StopIteration
    

class ChagePointMean(ChangePointSequence):
    '''
    Потоковая генерация случайной последовательности с разладкой в М.О.
    '''
    def __init__(self, N, mean, standard_deviation, change_point_position, mean_change):
        super().__init__(N, mean, standard_deviation, change_point_position)
        self.mean_change = mean_change
        
    def __next__(self):
        if self.counter < self.N:
            
            self.counter += 1
            
            new_element_sequence = float(norm.rvs(loc=self.mean, scale=self.standard_deviation, size=1)) \
            if self.counter <= self.change_point_position \
            else float(norm.rvs(loc=(self.mean+self.mean_change), scale=self.standard_deviation, size=1))
            
            self.sequence.append(new_element_sequence)
            return new_element_sequence
        
        else:
            raise StopIteration
            
            
class ChagePointMeanDynamic(ChangePointSequence):
    '''
    Потоковая генерация случайной последовательности с разладкой в М.О.
    '''
    def __init__(self, N, mean, standard_deviation, change_point_position, mean_change):
        super().__init__(N, mean, standard_deviation, change_point_position)
        self.mean_change = mean_change
        
    def __next__(self):
        if self.counter < self.N:
            
            self.counter += 1
            
            new_element_sequence = float(norm.rvs(loc=self.mean, scale=self.standard_deviation, size=1)) \
            if self.counter <= self.change_point_position \
            else float(norm.rvs(loc=(self.mean+self.mean_change*(self.counter-self.change_point_position)),
                                scale=self.standard_deviation,
                                size=1))
            
            self.sequence.append(new_element_sequence)
            return new_element_sequence
            
        else:
            raise StopIteration
            
            
class ChagePointDispersion(ChangePointSequence):
    '''
    Потоковая генерация случайной последовательности с разладкой в дисперсии
    '''
    def __init__(self, N, mean, standard_deviation, change_point_position, standard_deviation_change):
        super().__init__(N, mean, standard_deviation, change_point_position)
        self.standard_deviation_change = standard_deviation_change
        
    def __next__(self):
        if self.counter < self.N:
            
            self.counter += 1
            
            new_element_sequence = float(norm.rvs(loc=self.mean, scale=self.standard_deviation, size=1)) \
            if self.counter <= self.change_point_position \
            else float(norm.rvs(loc=self.mean, scale=(self.standard_deviation+self.standard_deviation_change), size=1))
            
            self.sequence.append(new_element_sequence)
            return new_element_sequence
        
        else:
            raise StopIteration    

# Алгоритм ГРШ

In [14]:
def algorithm_grsh(seq, g=12, k=100):
    R = [0]
    
    # Заполняем изначальными значениями
    while len(sequence.sequence) < k:
        new_value = next(sequence)

    from numpy import mean, std
    m = mean(sequence.sequence)
    s = np.std(sequence.sequence)
    v = (abs(m) + 2*s) 
    
    for new_value in seq:
        Rt = exp(new_value - v) * (1 + R[-1])
        if Rt > g:
            #print('changepoint', seq.counter - 1, 'Rt =', Rt)
            return seq.counter - 1
        R.append(Rt)
    #print('No change points.')
    return -1

In [19]:
sequence = ChagePointMeanDynamic(N=1000, mean=1, standard_deviation=1, change_point_position=400, mean_change=1/2)

algorithm_grsh(sequence, 12, 100)

401

# Алгоритм Воробейчикова

In [21]:
# Алгоритм Воробейчикова
def algorithm_change_point_St(seq, difference_step_count=10, t0=300, m=1, N=8):

    def z_n(N, m, x, t, t0):
        def sign(x):
            return 1 if x >= 0 else -1

        def y_n(x, t, t0):
            return sign(x[t] - x[t-t0])

        return N * y_n(x, t, t0) - m
    
    def dcp(difference_step_count, difference):
        if len(difference) < difference_step_count:
            return False
        previous = None
        for i in range(difference_step_count-1):
            if difference[-1-i] != difference[-2-i]:
                return False
        return True
    
    
    l = N + m
    Sn = []
    t = 0
    x = []
    difference = []
    
    while t <= t0:
        x.append(next(sequence))
        Sn.append(l)
        t += 1

    for new_value in sequence:
        x.append(new_value)
        zn = z_n(N, m, x, t, t0)
        current_Sn = max(Sn[-1] + zn, l)
        difference.append((current_Sn - Sn[-1]))
        if dcp(difference_step_count, difference):
            return (t, t - difference_step_count)

        Sn.append(current_Sn)
        t += 1
        
    return (-1, -1)

In [23]:
sequence = ChagePointMean(N=1000, mean=1, standard_deviation=1, change_point_position=400, mean_change=2)

algorithm_change_point_St(sequence, 10)[0]

410

# Алгоритм CUSUM bootstrap

In [75]:
def algorithm_CUSUM_bootstrap(seq, confidence_level=95, number_bootstrap_samples=1000):
    for i in seq:
        pass
    sequence = seq.sequence
    
    mean_y = np.mean(sequence)

    CUSUM = np.asarray([yi - mean_y for yi in sequence])
    CUSUM = np.cumsum(CUSUM)

    CUSUM_max = np.max(CUSUM)
    CUSUM_min = np.min(CUSUM)
    CUSUM_diff = CUSUM_max - CUSUM_min

    count = 0
    for i in range(number_bootstrap_samples):
        new_y = np.random.permutation(sequence)
        new_CUSUM = np.asarray([yi - mean_y for yi in new_y])
        new_CUSUM = np.cumsum(new_CUSUM)

        new_CUSUM_max = np.max(new_CUSUM)
        new_CUSUM_min = np.min(new_CUSUM)
        new_CUSUM_diff = new_CUSUM_max - new_CUSUM_min
        if new_CUSUM_diff < CUSUM_diff:
            count += 1

    confidence_level_calc = (count / number_bootstrap_samples) * 100
    if confidence_level_calc > confidence_level:
        abs_CUSUM = np.fabs(CUSUM)
        CUSUM_max = np.max(abs_CUSUM)
        change_point = int(np.where(abs_CUSUM == CUSUM_max)[0]) 
#         print('Точка разладки:', change_point)
        return change_point
    else: 
        return -1

In [199]:
sequence = ChagePointMean(N=800, mean=1, standard_deviation=5, change_point_position=300, mean_change=2)

algorithm_CUSUM_bootstrap(sequence, 90, 1000)


276

# Алгоритм Боровкова

In [65]:
def algorithm_change_point_Q(seq):
    for i in seq:
        pass
    
    f_1 = seq.sequence
    S_1_m = np.cumsum(f_1)
    
    f_2 = [y ** 2 for y in f_1]
    S_2_m = np.cumsum(f_2)
    
    Q1 = []
    Q2 = []
    n = len(f_1)
    
    for t in range(n):
        sum_Q = 0
        for i in range(0, t+1):
            sum_Q += (f_1[i] - S_1_m[t]/t) ** 2
            
        for i in range(t+1, n):
            sum_Q += (f_1[i] - (S_1_m[n-1]-S_1_m[t])/(n-t)) ** 2
        
        Q1.append(sum_Q)
        
    for t in range(n):
        sum_Q = 0
        for i in range(0, t+1):
            sum_Q += (f_2[i] - S_2_m[t]/t) ** 2
            
        for i in range(t+1, n):
            sum_Q += (f_2[i] - (S_2_m[n-1]-S_2_m[t])/(n-t)) ** 2
        
        Q2.append(sum_Q)
        
    Q = [q1+q2 for q1,q2 in zip(Q1,Q2)]
    
    min_Q = np.min(Q)
    change_point = int(np.where(Q == min_Q)[0]) # +1 ?
    return change_point
    

In [102]:
%%time
sequence = ChagePointMeanDynamic(N=600, mean=1, standard_deviation=1, change_point_position=300, mean_change=4)

algorithm_change_point_Q(sequence)

C:\Users\TryUserPWNZ\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\TryUserPWNZ\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars


Wall time: 1.61 s


475

# Алгоритм, основанный на информационном подходе

In [61]:
def algorithm_fa(seq, type_cp):
    for i in seq:
        pass
    sequence = seq.sequence
    
    n = len(sequence)
    tau = np.arange(1,n)
    lmbd = 2*np.log(n) #Bayesian Information Criterion
    eps = 1.e-8 #to avoid zeros in denominator
    
    if type_cp =="mean":
        mu0 = np.mean(sequence)
        s0 = np.sum((sequence-mu0)**2)
        s1 = np.asarray([np.sum((sequence[0:i]-np.mean(sequence[0:i]))**2) for i in range(1,n)])
        s2 = np.asarray([np.sum((sequence[i:]-np.mean(sequence[i:]))**2) for i in range(1,n)])
        R  = s0-s1-s2
        G  = np.max(R)
        taustar = int(np.where(R==G)[0]) 
        sd1 = np.std(sequence[0:taustar-1])
        sd2 = np.std(sequence[taustar-1:])
        var = ( taustar*sd1**2 + (n-taustar)*sd2**2 ) / n
        criterion = lmbd # var*lmbd
         
    elif type_cp =="var":
        std0 = np.std(sequence)
        std1 = np.asarray([np.std(sequence[0:i]) for i in range(1,n)],dtype=float) + eps
        std2 = np.asarray([np.std(sequence[i:]) for i in range(1,n)],dtype=float) + eps
        R = n*np.log(std0) - tau*np.log(std1) - (n-tau)*np.log(std2)
        G  = np.max(R)
        taustar = int(np.where(R==G)[0])
        criterion = lmbd
        
    if 2*G >= criterion:
        return taustar
    else:
        return -1
